<a href="https://colab.research.google.com/github/dev-aprilchoi/projectManager/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC0921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 웹 스크래핑

### requests 실습
- 위키백과에 요청을 보내고 res 변수에 웹사이트 응답을 받아옵니다.

In [ ]:
import requests
url = 'https://ko.wikipedia.org/wiki/HTML'
res = requests.get(url)
res

<Response [200]>

### Python에서 css 선택자 사용
설치

In [ ]:
!pip install cssselect

In [ ]:
res.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-disabled" lang="ko" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>HTML - 위키백과, 우리 모두의 백과사전</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-c

In [ ]:
import lxml.html
root = lxml.html.fromstring(res.text)
root

<Element html at 0x7d3d26fe8220>

In [ ]:
ts = root.cssselect('title')
ts[0].text_content()

'HTML - 위키백과, 우리 모두의 백과사전'

## 다음 뉴스 스크래핑: 검색 결과
왜 네이버말고 다음을 사용하나?

네이버는 많이 스크랩하면 하루정도 차단당함..ㅋ

문제: 매 사이트마다 CSS 디자인이 다르기 때문에
내용을 추출하기가 어렵다.

유명 언론사는 다음에서 돈 주고 기사를 사왔기 때문에

다음 내부에서 볼 수 있음

영세한 언론사는 다음에서 돈 주고 기사를 사오지 않기 때문에

그 언론사 홈페이지로 넘어감...ㅋㅋㅋㅋㅋ

-> 장단점: 유명하지 않은 언론사 기사는 퀄리티가 별로 좋지 않기때문에

필터링이 된다

In [ ]:
# 다음 사이트에 접속
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&p={}'
res = requests.get(url.format(1)) # 1 page로 가라
# HTML 처리
root = lxml.html.fromstring(res.text)
for link in root.cssselect('span.cont_info a.f_nb'):
    text = link.text_content().strip() # strip() : 앞뒤 공백 없애기
    if text == '다음뉴스':
        print(link.attrib['href'])

http://v.daum.net/v/20230919132700610
http://v.daum.net/v/20230920180621489
http://v.daum.net/v/20230920190011686
http://v.daum.net/v/20230920225911543
http://v.daum.net/v/20230919120618559
http://v.daum.net/v/20230921060909465
http://v.daum.net/v/20230918090503295
http://v.daum.net/v/20230921100311905


In [ ]:
# 기사 주소 수집
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=인공지능&p={}'

href = []
for page in range(1, 3): # 1~2 페이지 까지
    res = requests.get(url.format(page))
    root = lxml.html.fromstring(res.text)
    for link in root.cssselect('span.cont_info a.f_nb'):
        text = link.text_content().strip()
        if text == '다음뉴스':
            href.append(link.attrib['href'])

In [ ]:
# 기사 하나 본문 스크랩
res = requests.get('http://v.daum.net/v/20230919132700610')
root = lxml.html.fromstring(res.text)
article = root.cssselect('div.article_view')
article[0].text_content()

' \n         \n          \n           \n           \n            경기도가 챗지피티(ChatGPT) 같은 생성형 인공지능(AI)을 도정에 속속 도입하는 가운데 한계점 등을 두려워 말고 활용하기 시작해 위기를 기회로 만들어야 한다는 의견이 나왔다. 사진은 기업토론 모습. / 사진제공=경기도\n           \n          \n          \n           경기도가 챗지피티(ChatGPT) 같은 생성형 인공지능(AI)을 도정에 속속 도입하는 가운데 한계점 등을 두려워 말고 활용하기 시작해 위기를 기회로 만들어야 한다는 의견이 나왔다.\n          \n          경기도와 경기콘텐츠진흥원은 지난 14일부터 15일까지 수원컨벤션센터에서 \'인공지능이 확장한 문화기술의 세계\'를 주제로 \'2023년 문화기술 콘퍼런스\'를 열었다. 이번 행사는 챗지피티(ChatGPT) 같은 생성형 인공지능이 콘텐츠 산업계에 불러온 혁신을 조명하고 문화기술의 미래를 전망하기 위해 마련됐다.\n          이번 행사에는 \'인공지능의 미래\', \'인간은 필요없다\' 등을 저술한 제리 카플란(Jerry Kaplan) 교수와 유럽 인공지능 윤리학의 거두이자 \'특이점의 신화\'의 저자 장가브리엘 가나시아(Jean-Gabriel Ganascia) 소르본느 대학 교수 등 전 세계에서 저명한 석학들이 참석해 눈길을 끌었다.\n          장가브리엘 가나시아 소르본느 대학 교수는 경기도의 인공지능 도정 활용에 대해 "우선 기계를 사용해서 행정을 하고 있다는 것을 반드시 밝혀야 한다. 개인정보 침해를 조심해야 하고, 오답을 마치 정답처럼 말하는 환각(Hallucination) 문제를 경계해야 한다"며 "기계가 자동으로 무엇을 계속 생성하더라도 모든 것을 맡기지 말고, 대응 인력을 제대로 활용해야 한다"고 조언했다.\n          그러면서 "생성형 인공지능을 두려워하기보다 이용할 수 있어야 한다"며 "

In [ ]:
# 기사 여러개 본문 스크랩
import tqdm.notebook    # tqdm:진행 상태 막대바로 보여주는 기능
articles = []
for h in tqdm.notebook.tqdm(href):
    res = requests.get(h)
    root = lxml.html.fromstring(res.text)
    for article in root.cssselect('div.article_view'):
        articles.append(article.text_content())

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
articles

[' \n         \n          \n           \n           \n            경기도가 챗지피티(ChatGPT) 같은 생성형 인공지능(AI)을 도정에 속속 도입하는 가운데 한계점 등을 두려워 말고 활용하기 시작해 위기를 기회로 만들어야 한다는 의견이 나왔다. 사진은 기업토론 모습. / 사진제공=경기도\n           \n          \n          \n           경기도가 챗지피티(ChatGPT) 같은 생성형 인공지능(AI)을 도정에 속속 도입하는 가운데 한계점 등을 두려워 말고 활용하기 시작해 위기를 기회로 만들어야 한다는 의견이 나왔다.\n          \n          경기도와 경기콘텐츠진흥원은 지난 14일부터 15일까지 수원컨벤션센터에서 \'인공지능이 확장한 문화기술의 세계\'를 주제로 \'2023년 문화기술 콘퍼런스\'를 열었다. 이번 행사는 챗지피티(ChatGPT) 같은 생성형 인공지능이 콘텐츠 산업계에 불러온 혁신을 조명하고 문화기술의 미래를 전망하기 위해 마련됐다.\n          이번 행사에는 \'인공지능의 미래\', \'인간은 필요없다\' 등을 저술한 제리 카플란(Jerry Kaplan) 교수와 유럽 인공지능 윤리학의 거두이자 \'특이점의 신화\'의 저자 장가브리엘 가나시아(Jean-Gabriel Ganascia) 소르본느 대학 교수 등 전 세계에서 저명한 석학들이 참석해 눈길을 끌었다.\n          장가브리엘 가나시아 소르본느 대학 교수는 경기도의 인공지능 도정 활용에 대해 "우선 기계를 사용해서 행정을 하고 있다는 것을 반드시 밝혀야 한다. 개인정보 침해를 조심해야 하고, 오답을 마치 정답처럼 말하는 환각(Hallucination) 문제를 경계해야 한다"며 "기계가 자동으로 무엇을 계속 생성하더라도 모든 것을 맡기지 말고, 대응 인력을 제대로 활용해야 한다"고 조언했다.\n          그러면서 "생성형 인공지능을 두려워하기보다 이용할 수 있어야 한다"며 

In [ ]:
# 스크래핑 결과를 엑셀 파일 형태로 저장하기
import pandas as pd
news = pd.DataFrame({'주소': href, '본문': articles})
news.to_excel('기사.xlsx')

In [ ]:
# 제목까지 같이 긁어오기
import tqdm.notebook
titles = []
articles = []
for h in tqdm.notebook.tqdm(href):
    res = requests.get(h)
    root = lxml.html.fromstring(res.text)
    ta = zip(root.cssselect('h3.tit_view'),
             root.cssselect('div.article_view'))
    for title, article in ta:
        titles.append(title.text_content())
        articles.append(article.text_content())

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
# 파일 저장
import pandas as pd
news = pd.DataFrame({'주소': href, '제목': titles, '본문': articles})
news.to_excel('기사_제목포함.xlsx')

In [26]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 58.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=c0112ea2e778b03178102dc6039a0527a2b59210e5df9f7e612a98a8b6b71339
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [27]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
kiwi.add_user_word('인공지능', 'NNG')

True

In [28]:
def extract_nouns(text):
    for token in kiwi.tokenize(text):
        if token.tag in {'NNG', 'NNP'}:
            yield token.form

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=100, tokenizer=extract_nouns)
dtm = cv.fit_transform(news['본문'])
word_count = pd.DataFrame({
    '단어': cv.get_feature_names_out(),
    '빈도': dtm.sum(axis=0).flat
})
word_count.sort_values('빈도', ascending=False).head(20)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,단어,빈도
70,인공지능,125
15,기술,102
92,표준,70
87,특허,52
98,활용,34
46,산업,33
64,유망,32
88,특허청,32
77,정보,27
67,이야기,27


# 텍스트 지도 학습
## 감성 분석
- 문장 감성(sentiment)을 예측
    - 감성: 긍/부정
    - 찬성/반대, 좋다/싫다 등
- 기쁨, 슬픔, 분노 등은 정서(emotion)라고 함

- 방법:
     - 사전 기반
     - 머신 러닝 기반

요즘에는 머신 러닝을 이용해서 감성 분석을 한다

### 감성분석 실습: 전처리

In [32]:
# 파일 열기
import pandas as pd
df = pd.read_excel('yelp.xlsx')
df.head()

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [33]:
# 문서 단어 행렬 만들기
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, stop_words='english')
dtm = cv.fit_transform(df.review)

In [34]:
# x와 y를 지정
x = dtm
y = df.sentiment

In [35]:
# 데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2, # 20%의 데이터를 테스트용으로 유보
    random_state=42) # 유사난수의 씨앗값 seed을 42로 설정

### 사이킷런에서 로지스틱 회귀분석

In [36]:
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(
    penalty='elasticnet', solver='saga', random_state=42,   # 여기서는 epoch는 하지 않는다(자동)
    Cs=[0.1, 1, 10], l1_ratios=[0, 0.5, 1], max_iter=4000)
model.fit(x_train, y_train)

LogisticRegressionCV(Cs=[0.1, 1, 10], l1_ratios=[0, 0.5, 1], max_iter=4000,
                     penalty='elasticnet', random_state=42, solver='saga')

In [37]:
model.C_ # 가장 좋은 C

array([1.])

In [39]:
model.l1_ratio_ # 가장 좋은 L1의 비율

array([0])

In [40]:
model.score(x_train, y_train)  # 정확도가 높은편인데

0.94125

In [41]:
model.score(x_test, y_test) # 테스트 결과의 정확도는 그닥 높지 않은것을 확인할 수 있다

0.76

In [42]:
# 단어별 가중치
word_coef = pd.DataFrame({
'단어': cv.get_feature_names_out(),
'가중치': model.coef_.flat
})

In [43]:
# 긍정 단어
word_coef.sort_values('가중치').tail(10)

,단어,가중치
208,excellent,1.340621
221,fantastic,1.439516
250,friendly,1.524526
364,love,1.525609
265,good,1.532660
409,nice,1.555230
32,awesome,1.714438
12,amazing,1.779166
153,delicious,2.201475
268,great,2.867876


In [44]:
# 부정 단어
word_coef.sort_values('가중치').head(10)

,단어,가중치
37,bad,-1.448292
179,don,-1.397359
65,bland,-1.274375
939,wasn,-1.225014
392,minutes,-1.177965
977,worst,-1.146635
607,rude,-1.040134
889,unfortunately,-1.039067
703,slow,-1.036536
546,probably,-1.015125
